In [ ]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")

using LinearAlgebra # dla norm macierzy

setprecision(64)

function randSquareMatrix(n::Int64)
    return BigFloat.(rand(n, n))
end

# f - Inversion method
# n - Dimensions of matrices
function testMethod(f::Function; tests=1000::Int64, n=2:10, norm=(x -> norm(x, 2)), log=true)
    results_mean = BigFloat.(zeros(length(n)))
    results_max = BigFloat.(zeros(length(n)))
    Σ = BigFloat(0)
    i = 1
    for size = n
        for t = 1:tests
            m = randSquareMatrix(size)
            res = norm(m * f(m) - BigFloat.(eye(size)))
            Σ += res
            if res > results_max[i]
                results_max[i] = res
            end
        end
        results_mean[i] = Σ / tests
        if log
            @printf "Matrix size: %3d Tests: %3d Mean error: %.4e Max error: %.4e" size tests results_mean[i] results_max[i]
        end
        i += 1
    end
    return results, results_max
end

function LUDecomposition(m::Matrix{BigFloat})
    n = size(m)[1]
    u = Matrix{BigFloat}(n, n)
    l = Matrix{BigFloat}(n, n)
    u[1, 1] = m[1, 1]
    l[1, 1] = 1
    for j = 2:n
        u[1, j] = m[1, j]
        l[j, 1] = m[j, 1] / u[1, 1]
    end
    for i = 2:n
        for j = 1:i-1
            u[i, j] = 0
            l[j, i] = 0
        end
        u[i, i] = m[i, i] - sum([l[i, k] * u[k, i] for k = 1:i-1])
        l[i, i] = 1
        for j = i+1:n
            u[i, j] = m[i, j] - sum([l[i, k] * u[k, j] for k = 1:i-1])
            l[j, i] = (m[j, i] - sum([l[j, k] * u[k, i] for k = 1:i-1])) / u[i, i]
        end
    end
    return l, u
end

function Gauss(A::Matrix{BigFloat})
    n = size(A)[1]
    v = BigFloat.(eye(n))
    m = hcat(A, v)
    for i = 1:n
        p = sortperm(abs.(m[i:end, i]), rev=true) + i - 1
        m[i:end, :] = m[p, :]
        m[i, :] /= m[i, i]
        for j = i+1:n
            a = m[j, i]
            b = m[i, i]
            m[j, :] *= b
            m[j, :] -= a * m[i, :]
            m[j, :] /= b
        end
    end
    for i = n:-1:2
        for j = 1:i-1
            a = m[j, i]
            m[j, :] -= a * m[i, :]
        end
    end
    return m[:, n+1:end]
end

function makeInverse(f::Function, A::Matrix{BigFloat}; A⁻¹::Matrix{BigFloat}=zeros(A), max_iter=Inf)
    n = size(A)[1]
    B = BigFloat.(zeros(n, n))
    E = BigFloat.(eye(n))
    for i = 1:n
        B[:, i] = f(A, E[:, i], A⁻¹[:, i], iterations=max_iter)
    end
    return B
end 

function makeCleverInverse(f::Function, A::Matrix{BigFloat}; A⁻¹::Matrix{BigFloat}=zeros(A), max_iter=Inf)
    n = size(A)[1]
    B = BigFloat.(zeros(n, n))
    E = copy(A')
    for i = 1:n
        B[:, i] = f(A' * A, E[:, i], A⁻¹[:, i], iterations=max_iter)
    end
    return B
end

testMethod(inv)

Matrix size: 

In [ ]:
function Richardson(A::Matrix{BigFloat}, b::Vector{BigFloat}, x::Vector{BigFloat}; τ=3, iterations=Inf)
    i = 0
    while i < iterations
        δ = - τ * A * x + τ * b
        if norm(δ) == 0
            i = Inf
        end
        x += δ
        i += 1
    end
    return x
end

function SOR(A::Matrix{BigFloat}, b::Vector{BigFloat}, x::Vector{BigFloat}; ω=1, iterations=Inf)
    t = 0
    n = size(A)[1]
    while t < iterations
        change = 0
        for i = 1:n
            σ = 0
            for j = 1:n
                if i != j
                    σ += A[i, j] * x[j]
                end
            end
            δ = ω * ((b[i] - σ) / A[i, i] - x[i])
            x[i] += δ
            change += δ
        end
        if change == 0
            @printf "SOR iterations: %d\n" t
            t = Inf
        end
        t += 1
    end
    return x
end

n = 10
A = randSquareMatrix(n)
b = BigFloat.(rand(n))
#A = B * B'
A_1 = Gauss(A)
A_2 = makeCleverInverse(SOR, A, A⁻¹=A_1)
A_3 = makeInverse(SOR, A, A⁻¹=A_1, max_iter=100)
@show norm(A * A_1 - eye(n))
@show norm(A * A_2 - eye(n))
@show norm(A * A_3 - eye(n))

In [ ]:
function BiCGSTAB(A::Matrix{BigFloat}, b::Vector{BigFloat}, x::Vector{BigFloat}; iterations=Inf)
    r = b - A * x
    R = r
    ρ = α = ω = 1
    v = p = 0
    i = 1
    while i < iterations
        ρ = dot(R, r)
        
        i += 1
    end
end